# Strategy

Plan of Attack

หลักการของกลยุทธ์

การจำลองระบบการซื้อขาย (Trade Execution Simulation)

Strategy Implementation & Backtesting

# Code

# UXUI

In [1]:
import pandas as pd
import ipywidgets as widgets
import os
from IPython.display import display, clear_output

def create_csv_viewer(folder_path):
    """
    📌 ฟังก์ชันสำหรับสร้าง UI ดูไฟล์ CSV ในโฟลเดอร์ที่ระบุ
    folder_path: str -> โฟลเดอร์ที่เก็บไฟล์ CSV
    """
    
    # 📌 ดึงรายชื่อไฟล์ทั้งหมดจากโฟลเดอร์
    file_list = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

    # 📌 ตัวเลือกเลือกไฟล์และจำนวนแถว
    file_picker = widgets.Dropdown(
        options=file_list,
        value=file_list[0] if file_list else None,
        description="Select File:",
        style={'description_width': 'initial'}
    )

    rows_per_page = widgets.Dropdown(
        options=[5, 10, 20, 50, 100],
        value=10,
        description="Rows per Page:",
        style={'description_width': 'initial'}
    )

    # 📌 ตัวแปรเก็บข้อมูลของตาราง
    df = pd.DataFrame()
    current_page = 0

    # 📌 ปุ่มเปลี่ยนหน้า
    prev_button = widgets.Button(description="⏮ Previous", disabled=True)
    next_button = widgets.Button(description="Next ⏭")
    page_label = widgets.Label(value="Page 1")
    jump_to_page = widgets.IntText(value=1, description="Jump to Page:", min=1)
    jump_button = widgets.Button(description="Go")

    # 📌 พื้นที่แสดงข้อมูล
    output = widgets.Output()

    def load_data():
        """โหลดข้อมูลจากไฟล์ CSV ที่เลือก"""
        nonlocal df, current_page
        file_path = os.path.join(folder_path, file_picker.value)
        df = pd.read_csv(file_path)
        current_page = 0
        update_table()

    def update_table():
        """อัปเดตข้อมูลในตาราง"""
        with output:
            clear_output(wait=True)
            start = current_page * rows_per_page.value
            end = start + rows_per_page.value
            display(df.iloc[start:end])

            # 📌 อัปเดตหมายเลขหน้า
            total_pages = max(1, (len(df) - 1) // rows_per_page.value + 1)
            page_label.value = f"Page {current_page + 1} of {total_pages}"

            # 📌 อัปเดตค่าต่ำสุดสูงสุดของ Jump to Page
            jump_to_page.min = 1
            jump_to_page.max = total_pages

            # 📌 อัปเดตสถานะปุ่ม
            prev_button.disabled = current_page == 0
            next_button.disabled = end >= len(df)

    def prev_page(_):
        """ย้อนกลับไปหน้าก่อนหน้า"""
        nonlocal current_page
        if current_page > 0:
            current_page -= 1
            update_table()

    def next_page(_):
        """ไปยังหน้าถัดไป"""
        nonlocal current_page
        if (current_page + 1) * rows_per_page.value < len(df):
            current_page += 1
            update_table()

    def jump_to_selected_page(_):
        """ข้ามไปยังหน้าที่ต้องการ"""
        nonlocal current_page
        total_pages = max(1, (len(df) - 1) // rows_per_page.value + 1)
        if 1 <= jump_to_page.value <= total_pages:
            current_page = jump_to_page.value - 1
            update_table()

    # 📌 เชื่อมโยงปุ่มกับฟังก์ชัน
    prev_button.on_click(prev_page)
    next_button.on_click(next_page)
    jump_button.on_click(jump_to_selected_page)

    # 📌 อัปเดตเมื่อเลือกไฟล์หรือเปลี่ยนจำนวนแถว
    def on_change(change):
        load_data()

    file_picker.observe(on_change, names="value")
    rows_per_page.observe(on_change, names="value")

    # 📌 จัดวาง UI
    ui_top = widgets.HBox([file_picker, rows_per_page], layout=widgets.Layout(justify_content="center"))
    ui_bottom = widgets.HBox([
        prev_button, 
        page_label, 
        next_button, 
        jump_to_page, 
        jump_button
    ], layout=widgets.Layout(justify_content="flex-end"))

    display(ui_top, output, ui_bottom)
    load_data()



# MeanReversion - ReverseOrderFlowImbalance - Strategy 

In [2]:
import pandas as pd
import numpy as np
import os


file_paths = {
    "2024-05-02": "split_data/S50M24_2024-05-02.csv",
    "2024-05-03": "split_data/S50M24_2024-05-03.csv",
    "2024-05-07": "split_data/S50M24_2024-05-07.csv",
    "2024-05-08": "split_data/S50M24_2024-05-08.csv",
    "2024-05-09": "split_data/S50M24_2024-05-09.csv",
    "2024-05-10": "split_data/S50M24_2024-05-10.csv",
    "2024-05-13": "split_data/S50M24_2024-05-13.csv",
    "2024-05-14": "split_data/S50M24_2024-05-14.csv",
}

# 📌 โฟลเดอร์เก็บ Log
log_folder = "Trade_Logs"
transaction_folder = "Transaction_Pairs"
os.makedirs(log_folder, exist_ok=True)
os.makedirs(transaction_folder, exist_ok=True)

# 📌 ฟังก์ชันสำหรับไล่ระดับราคา (Market Depth Matching)
def get_vwap_and_levels(volume_needed, price_levels, volume_levels, side):
    total_value = 0
    total_size = 0
    used_levels = []

    for i in range(len(price_levels)):
        price = price_levels[i]
        available_volume = volume_levels[i]

        if available_volume <= 0:
            continue

        size_to_use = min(volume_needed, available_volume)
        total_value += size_to_use * price
        total_size += size_to_use
        used_levels.append(side + str(i + 1))

        volume_needed -= size_to_use
        if volume_needed <= 0:
            break

    vwap_price = total_value / total_size if total_size > 0 else 0
    return vwap_price, used_levels

# 📌 ฟังก์ชันสำหรับประมวลผลไฟล์
def process_file(file_path, date):
    df = pd.read_csv(file_path, low_memory=False)

    if "timestamp" in df.columns:
        df.rename(columns={"timestamp": "Timestamp"}, inplace=True)

    required_columns = {"BidTrade", "AskTrade", "BidVolume", "AskVolume", "Bid1", "Ask1", "Timestamp",
                        "vBid1", "vAsk1", "Bid2", "vBid2", "Bid3", "vBid3", "Ask2", "vAsk2", "Ask3", "vAsk3"}
    if missing_columns := required_columns - set(df.columns):
        print(f"❌ Skipping {file_path} - Missing columns: {missing_columns}")
        return None

    df.fillna({"BidTrade": 0, "AskTrade": 0, "BidVolume": 0, "AskVolume": 0}, inplace=True)

    df["OFI"] = (df["BidTrade"] * df["BidVolume"]) - (df["AskTrade"] * df["AskVolume"])
    q80, q20 = df["OFI"].quantile(0.8), df["OFI"].quantile(0.2)

    df["Signal"] = 0
    df.loc[df["OFI"] > q80, "Signal"] = 1
    df.loc[df["OFI"] < q20, "Signal"] = -1

    log_file_path = os.path.join(log_folder, f"Trade_Log_{date}.csv")
    transaction_file_path = os.path.join(transaction_folder, f"Transaction_Pair_{date}.csv")
    
    log_data = []
    transaction_data = []

    open_long_positions = []
    open_short_positions = []
    commission_rate = 0.00007
    initial_capital = 1_000_000
    capital = initial_capital
    cash = initial_capital
    peak_capital = initial_capital
    total_fees = 0

    for row in df.itertuples(index=False):
        signal, timestamp = row.Signal, row.Timestamp

        bid_prices, bid_volumes = [row.Bid1, row.Bid2, row.Bid3], [row.vBid1, row.vBid2, row.vBid3]
        ask_prices, ask_volumes = [row.Ask1, row.Ask2, row.Ask3], [row.vAsk1, row.vAsk2, row.vAsk3]

        if signal == 1:
            if open_short_positions:
                for pos in open_short_positions:
                    vwap_price, used_levels = get_vwap_and_levels(pos["Size"], bid_prices, bid_volumes, "Bid")
                    profit = (pos["EntryPrice"] - vwap_price) * pos["Size"]
                    fee_close = pos["Size"] * vwap_price * commission_rate
                    net_profit = profit - (pos["Fee"] + fee_close)
                    capital += profit - fee_close
                    cash -= pos["Size"] * vwap_price + fee_close
                    total_fees += fee_close
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    log_data.append([timestamp, "CLOSE_SHORT", pos["Size"], vwap_price, fee_close, profit, used_levels, vwap_price, capital, cash, drawdown])
                    transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee_close, profit, net_profit])

                open_short_positions.clear()
            else:
                size = 0.3 * row.vBid1
                vwap_price, used_levels = get_vwap_and_levels(size, bid_prices, bid_volumes, "Bid")
                fee = size * vwap_price * commission_rate

                if cash >= size * vwap_price + fee:
                    cash -= size * vwap_price + fee
                    capital -= fee
                    total_fees += fee
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    open_long_positions.append({"Size": size, "EntryPrice": vwap_price, "Timestamp": timestamp, "Fee": fee})
                    log_data.append([timestamp, "OPEN_LONG", size, vwap_price, fee, 0, used_levels, vwap_price, capital, cash, drawdown])

        elif signal == -1:
            if open_long_positions:
                for pos in open_long_positions:
                    vwap_price, used_levels = get_vwap_and_levels(pos["Size"], ask_prices, ask_volumes, "Ask")
                    profit = (vwap_price - pos["EntryPrice"]) * pos["Size"]
                    fee_close = pos["Size"] * vwap_price * commission_rate
                    net_profit = profit - (pos["Fee"] + fee_close)
                    capital += profit - fee_close
                    cash += pos["Size"] * vwap_price - fee_close
                    total_fees += fee_close
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    log_data.append([timestamp, "CLOSE_LONG", pos["Size"], vwap_price, fee_close, profit, used_levels, vwap_price, capital, cash, drawdown])
                    transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee_close, profit, net_profit])

                open_long_positions.clear()
            else:
                size = 0.3 * row.vAsk1
                vwap_price, used_levels = get_vwap_and_levels(size, ask_prices, ask_volumes, "Ask")
                fee = size * vwap_price * commission_rate

                if cash >= size * vwap_price + fee:
                    cash += size * vwap_price - fee  # เปิด Short ต้องได้เงินสดเข้า
                    capital -= fee
                    total_fees += fee
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    open_short_positions.append({"Size": size, "EntryPrice": vwap_price, "Timestamp": timestamp, "Fee": fee})
                    log_data.append([timestamp, "OPEN_SHORT", size, vwap_price, fee, 0, used_levels, vwap_price, capital, cash, drawdown])

    # ✅ บังคับปิดสถานะทั้งหมดก่อนหมดวัน
    if open_long_positions or open_short_positions:
        final_row = df.iloc[-1]
        bid_prices = [final_row.Bid1, final_row.Bid2, final_row.Bid3]
        bid_volumes = [final_row.vBid1, final_row.vBid2, final_row.vBid3]

        ask_prices = [final_row.Ask1, final_row.Ask2, final_row.Ask3]
        ask_volumes = [final_row.vAsk1, final_row.vAsk2, final_row.vAsk3]

        # ✅ บังคับปิด LONG
        for pos in open_long_positions:
            vwap_price, used_levels = get_vwap_and_levels(pos["Size"], ask_prices, ask_volumes, "Ask")
            profit = (vwap_price - pos["EntryPrice"]) * pos["Size"]
            fee = pos["Size"] * vwap_price * commission_rate
            net_profit = profit - (pos["Fee"] + fee)

            capital += profit - fee
            cash += pos["Size"] * vwap_price - fee  # เงินสดกลับเข้า Cash
            total_fees += fee
            peak_capital = max(peak_capital, capital)
            drawdown = peak_capital - capital

            log_data.append([timestamp, "FORCED_CLOSE_LONG", pos["Size"], vwap_price, fee, profit, used_levels, vwap_price, capital, cash, drawdown])
            transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee, profit, net_profit])

        # ✅ บังคับปิด SHORT
        for pos in open_short_positions:
            vwap_price, used_levels = get_vwap_and_levels(pos["Size"], bid_prices, bid_volumes, "Bid")
            profit = (pos["EntryPrice"] - vwap_price) * pos["Size"]
            fee = pos["Size"] * vwap_price * commission_rate
            net_profit = profit - (pos["Fee"] + fee)

            capital += profit - fee
            cash -= pos["Size"] * vwap_price + fee  # หักเงินสดออกตอนปิด Short
            total_fees += fee
            peak_capital = max(peak_capital, capital)
            drawdown = peak_capital - capital

            log_data.append([timestamp, "FORCED_CLOSE_SHORT", pos["Size"], vwap_price, fee, profit, used_levels, vwap_price, capital, cash, drawdown])
            transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee, profit, net_profit])
   
    # ✅ คำนวณ Maximum Drawdown (MDD)
    #df_log = pd.DataFrame(log_data, columns=["Timestamp", "Type", "Size", "Price", "Fee", "Profit", "Lavel", "VWAP", "Capital", "Drawdown"])
   
    
    df_log = pd.DataFrame(log_data, columns=["Timestamp", "Type", "Size", "Price", "Fee", "Profit", "Lavel", "VWAP", "Capital", "Cash", "Drawdown"])
    max_drawdown = df_log["Drawdown"].max()
    df_transaction = pd.DataFrame(transaction_data, columns=["Timestamp_Open", "Timestamp_Close", "Size", "Price_Open", "Price_Close", "Fee_Open", "Fee_Close", "Profit", "Net_Profit"])
    df_log.to_csv(log_file_path, index=False)
    df_transaction.to_csv(transaction_file_path, index=False)
    
    #Mean_return = df_log["Profit"].mean() 
    Mean_return = df_transaction["Net_Profit"].mean() 
    
    #std_return = df_log["Profit"].std()
    std_return = df_transaction["Net_Profit"].std()
    
    #total_profit = df_log[df_log["Profit"] > 0]["Profit"].sum()
    total_profit = df_transaction[df_transaction["Net_Profit"] > 0]["Net_Profit"].sum()
    
    #total_loss = df_log[df_log["Profit"] < 0]["Profit"].sum()
    total_loss = df_transaction[df_transaction["Net_Profit"] < 0]["Net_Profit"].sum()
    
    Sharpe_Ratio = Mean_return/std_return
    ProfitFactor = int(total_profit)/(int(total_loss)*(-1))
    Profit_count = df_log[df_log["Profit"] > 0]["Profit"].count() #แบบไม่หักค่าธรรมเนียม
    Profit_count = df_transaction[df_transaction["Net_Profit"] > 0]["Net_Profit"].count()
    Loss_count = df_log[df_log["Profit"] < 0]["Profit"].count() #แบบไม่หักค่าธรรมเนียม
    Loss_count = df_transaction[df_transaction["Net_Profit"] < 0]["Net_Profit"].count()
    Win_rate = (Profit_count)/(Profit_count+Loss_count) 
    
    Total_Trades = len(df_log)
    
    #Total_Capital_used = df_transaction["Size"] * df_transaction["Price_Open"]
    #Total_Capital_used = total_capital_used.sum()
    
    #Slippage Cost
    df_transaction["Slippage"] = abs(df_transaction["Price_Open"] - df_transaction["Price_Close"])
    Total_Slippage_Cost = df_transaction["Slippage"].sum()
    
    #Market Impact Cost 
    #df_transaction["Market_Impact"] = abs(df_transaction["Price_Open"] - df_transaction["VWAP"])
    #total_market_impact_cost = df_transaction["Market_Impact"].sum()
    #AVG
    #avg_market_impact = df_transaction["Market_Impact"].mean()
   
    
    #Commission & Fee Cost
    total_fee_cost = df_transaction["Fee_Open"].sum() + df_transaction["Fee_Close"].sum()

    #Drawdown Duration ไม่ถูก และยังนึกไม่ออกว่าจะใช้อะไรคำนวณ
    #df_log["Peak_Capital"] = df_log["Capital"].cummax()
    #df_log["Drawdown_Flag"] = df_log["Capital"] < df_log["Peak_Capital"]
    #drawdown_duration = df_log["Drawdown_Flag"].sum()



    
    return {"Date": date, 
            "Final PnL": round(capital - initial_capital, 2), 
            "ROI_IN (%)": round((capital - initial_capital) * 100 / initial_capital, 2), 
            "Final Capital": round(capital, 2), 
            "MDD": round(max_drawdown, 2),
            "Sharpe Ratio" : round(Sharpe_Ratio, 2) ,
            "Profit Factor" : round(ProfitFactor, 2),
            "Win-Rate " : round(Win_rate, 2),
            "Total Trades" : round(Total_Trades, 2),
            "Total Slippage Cost " : round(Total_Slippage_Cost, 2),
            #"Total Market Impact Cost " : round(total_market_impact_cost, 2),
            #"AVG Market Impact " : round(avg_market_impact, 2),
            "Total Fee Cost " : round(total_fee_cost, 2),
            #"Drawdown Duration " : round(drawdown_duration, 2),
           }

# 📌 ประมวลผล
df_results = pd.DataFrame([process_file(path, date) for date, path in file_paths.items()])
df_results.to_csv("Performance_Report.csv", index=False)
#df_results

In [3]:
df_results

,Date,Final PnL,ROI_IN (%),Final Capital,MDD,Sharpe Ratio,Profit Factor,Win-Rate,Total Trades,Total Slippage Cost,Total Fee Cost
0,2024-05-02,6766.69,0.68,1006766.69,289.45,0.17,2.57,0.31,29526,1741.66,28119.73
1,2024-05-03,9635.62,0.96,1009635.62,271.51,0.18,3.37,0.33,25604,1544.78,30297.78
2,2024-05-07,9380.96,0.94,1009380.96,617.47,0.15,2.46,0.28,39568,2318.65,42981.60
3,2024-05-08,5765.06,0.58,1005765.06,266.67,0.14,2.29,0.29,32506,1887.70,29643.14
4,2024-05-09,4172.57,0.42,1004172.57,428.49,0.11,1.80,0.27,32352,1830.20,30474.76
5,2024-05-10,3246.99,0.32,1003246.99,261.91,0.13,1.96,0.27,24218,1375.86,19307.22
6,2024-05-13,3440.72,0.34,1003440.72,219.87,0.12,1.82,0.31,25432,1495.10,22149.65
7,2024-05-14,7798.51,0.78,1007798.51,456.87,0.14,2.73,0.31,29198,1740.13,28527.07


In [4]:
create_csv_viewer("Trade_Logs")

Output()

In [5]:
create_csv_viewer("Transaction_Pairs")

Output()

# Blind Test Data

In [6]:
import pandas as pd
import numpy as np
import os

file_paths = {
    "2024-05-15": "split_data/S50M24_2024-05-15.csv",
    "2024-05-16": "split_data/S50M24_2024-05-16.csv",
}

# 📌 โฟลเดอร์เก็บ Log
log_folder = "Trade_Logs_Blind_Test"
transaction_folder = "Transaction_Pairs_Blind_Test"
os.makedirs(log_folder, exist_ok=True)
os.makedirs(transaction_folder, exist_ok=True)

# 📌 ฟังก์ชันสำหรับไล่ระดับราคา (Market Depth Matching)
def get_vwap_and_levels(volume_needed, price_levels, volume_levels, side):
    total_value = 0
    total_size = 0
    used_levels = []

    for i in range(len(price_levels)):
        price = price_levels[i]
        available_volume = volume_levels[i]

        if available_volume <= 0:
            continue

        size_to_use = min(volume_needed, available_volume)
        total_value += size_to_use * price
        total_size += size_to_use
        used_levels.append(side + str(i + 1))

        volume_needed -= size_to_use
        if volume_needed <= 0:
            break

    vwap_price = total_value / total_size if total_size > 0 else 0
    return vwap_price, used_levels

# 📌 ฟังก์ชันสำหรับประมวลผลไฟล์
def process_file(file_path, date):
    df = pd.read_csv(file_path, low_memory=False)

    if "timestamp" in df.columns:
        df.rename(columns={"timestamp": "Timestamp"}, inplace=True)

    required_columns = {"BidTrade", "AskTrade", "BidVolume", "AskVolume", "Bid1", "Ask1", "Timestamp",
                        "vBid1", "vAsk1", "Bid2", "vBid2", "Bid3", "vBid3", "Ask2", "vAsk2", "Ask3", "vAsk3"}
    if missing_columns := required_columns - set(df.columns):
        print(f"❌ Skipping {file_path} - Missing columns: {missing_columns}")
        return None

    df.fillna({"BidTrade": 0, "AskTrade": 0, "BidVolume": 0, "AskVolume": 0}, inplace=True)

    df["OFI"] = (df["BidTrade"] * df["BidVolume"]) - (df["AskTrade"] * df["AskVolume"])
    q80, q20 = df["OFI"].quantile(0.8), df["OFI"].quantile(0.2)

    df["Signal"] = 0
    df.loc[df["OFI"] > q80, "Signal"] = 1
    df.loc[df["OFI"] < q20, "Signal"] = -1

    log_file_path = os.path.join(log_folder, f"Trade_Log_{date}.csv")
    transaction_file_path = os.path.join(transaction_folder, f"Transaction_Pair_{date}.csv")
    
    log_data = []
    transaction_data = []

    open_long_positions = []
    open_short_positions = []
    commission_rate = 0.00007
    initial_capital = 1_000_000
    capital = initial_capital
    cash = initial_capital
    peak_capital = initial_capital
    total_fees = 0

    for row in df.itertuples(index=False):
        signal, timestamp = row.Signal, row.Timestamp

        bid_prices, bid_volumes = [row.Bid1, row.Bid2, row.Bid3], [row.vBid1, row.vBid2, row.vBid3]
        ask_prices, ask_volumes = [row.Ask1, row.Ask2, row.Ask3], [row.vAsk1, row.vAsk2, row.vAsk3]

        if signal == 1:
            if open_short_positions:
                for pos in open_short_positions:
                    vwap_price, used_levels = get_vwap_and_levels(pos["Size"], bid_prices, bid_volumes, "Bid")
                    profit = (pos["EntryPrice"] - vwap_price) * pos["Size"]
                    fee_close = pos["Size"] * vwap_price * commission_rate
                    net_profit = profit - (pos["Fee"] + fee_close)
                    capital += profit - fee_close
                    cash -= pos["Size"] * vwap_price + fee_close
                    total_fees += fee_close
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    log_data.append([timestamp, "CLOSE_SHORT", pos["Size"], vwap_price, fee_close, profit, used_levels, vwap_price, capital, cash, drawdown])
                    transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee_close, profit, net_profit])

                open_short_positions.clear()
            else:
                size = 0.3 * row.vBid1
                vwap_price, used_levels = get_vwap_and_levels(size, bid_prices, bid_volumes, "Bid")
                fee = size * vwap_price * commission_rate

                if cash >= size * vwap_price + fee:
                    cash -= size * vwap_price + fee
                    capital -= fee
                    total_fees += fee
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    open_long_positions.append({"Size": size, "EntryPrice": vwap_price, "Timestamp": timestamp, "Fee": fee})
                    log_data.append([timestamp, "OPEN_LONG", size, vwap_price, fee, 0, used_levels, vwap_price, capital, cash, drawdown])

        elif signal == -1:
            if open_long_positions:
                for pos in open_long_positions:
                    vwap_price, used_levels = get_vwap_and_levels(pos["Size"], ask_prices, ask_volumes, "Ask")
                    profit = (vwap_price - pos["EntryPrice"]) * pos["Size"]
                    fee_close = pos["Size"] * vwap_price * commission_rate
                    net_profit = profit - (pos["Fee"] + fee_close)
                    capital += profit - fee_close
                    cash += pos["Size"] * vwap_price - fee_close
                    total_fees += fee_close
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    log_data.append([timestamp, "CLOSE_LONG", pos["Size"], vwap_price, fee_close, profit, used_levels, vwap_price, capital, cash, drawdown])
                    transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee_close, profit, net_profit])

                open_long_positions.clear()
            else:
                size = 0.3 * row.vAsk1
                vwap_price, used_levels = get_vwap_and_levels(size, ask_prices, ask_volumes, "Ask")
                fee = size * vwap_price * commission_rate

                if cash >= size * vwap_price + fee:
                    cash += size * vwap_price - fee  # เปิด Short ต้องได้เงินสดเข้า
                    capital -= fee
                    total_fees += fee
                    peak_capital = max(peak_capital, capital)
                    drawdown = peak_capital - capital

                    open_short_positions.append({"Size": size, "EntryPrice": vwap_price, "Timestamp": timestamp, "Fee": fee})
                    log_data.append([timestamp, "OPEN_SHORT", size, vwap_price, fee, 0, used_levels, vwap_price, capital, cash, drawdown])

    # ✅ บังคับปิดสถานะทั้งหมดก่อนหมดวัน
    if open_long_positions or open_short_positions:
        final_row = df.iloc[-1]
        bid_prices = [final_row.Bid1, final_row.Bid2, final_row.Bid3]
        bid_volumes = [final_row.vBid1, final_row.vBid2, final_row.vBid3]

        ask_prices = [final_row.Ask1, final_row.Ask2, final_row.Ask3]
        ask_volumes = [final_row.vAsk1, final_row.vAsk2, final_row.vAsk3]

        # ✅ บังคับปิด LONG
        for pos in open_long_positions:
            vwap_price, used_levels = get_vwap_and_levels(pos["Size"], ask_prices, ask_volumes, "Ask")
            profit = (vwap_price - pos["EntryPrice"]) * pos["Size"]
            fee = pos["Size"] * vwap_price * commission_rate
            net_profit = profit - (pos["Fee"] + fee)

            capital += profit - fee
            cash += pos["Size"] * vwap_price - fee  # เงินสดกลับเข้า Cash
            total_fees += fee
            peak_capital = max(peak_capital, capital)
            drawdown = peak_capital - capital

            log_data.append([timestamp, "FORCED_CLOSE_LONG", pos["Size"], vwap_price, fee, profit, used_levels, vwap_price, capital, cash, drawdown])
            transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee, profit, net_profit])

        # ✅ บังคับปิด SHORT
        for pos in open_short_positions:
            vwap_price, used_levels = get_vwap_and_levels(pos["Size"], bid_prices, bid_volumes, "Bid")
            profit = (pos["EntryPrice"] - vwap_price) * pos["Size"]
            fee = pos["Size"] * vwap_price * commission_rate
            net_profit = profit - (pos["Fee"] + fee)

            capital += profit - fee
            cash -= pos["Size"] * vwap_price + fee  # หักเงินสดออกตอนปิด Short
            total_fees += fee
            peak_capital = max(peak_capital, capital)
            drawdown = peak_capital - capital

            log_data.append([timestamp, "FORCED_CLOSE_SHORT", pos["Size"], vwap_price, fee, profit, used_levels, vwap_price, capital, cash, drawdown])
            transaction_data.append([pos["Timestamp"], timestamp, pos["Size"], pos["EntryPrice"], vwap_price, pos["Fee"], fee, profit, net_profit])
   
    # ✅ คำนวณ Maximum Drawdown (MDD)
    #df_log = pd.DataFrame(log_data, columns=["Timestamp", "Type", "Size", "Price", "Fee", "Profit", "Lavel", "VWAP", "Capital", "Drawdown"])
   
    
    df_log = pd.DataFrame(log_data, columns=["Timestamp", "Type", "Size", "Price", "Fee", "Profit", "Lavel", "VWAP", "Capital", "Cash", "Drawdown"])
    max_drawdown = df_log["Drawdown"].max()
    df_transaction = pd.DataFrame(transaction_data, columns=["Timestamp_Open", "Timestamp_Close", "Size", "Price_Open", "Price_Close", "Fee_Open", "Fee_Close", "Profit", "Net_Profit"])
    df_log.to_csv(log_file_path, index=False)
    df_transaction.to_csv(transaction_file_path, index=False)
    
    #Mean_return = df_log["Profit"].mean() 
    Mean_return = df_transaction["Net_Profit"].mean() 
    
    #std_return = df_log["Profit"].std()
    std_return = df_transaction["Net_Profit"].std()
    
    #total_profit = df_log[df_log["Profit"] > 0]["Profit"].sum()
    total_profit = df_transaction[df_transaction["Net_Profit"] > 0]["Net_Profit"].sum()
    
    #total_loss = df_log[df_log["Profit"] < 0]["Profit"].sum()
    total_loss = df_transaction[df_transaction["Net_Profit"] < 0]["Net_Profit"].sum()
    
    Sharpe_Ratio = Mean_return/std_return
    ProfitFactor = int(total_profit)/(int(total_loss)*(-1))
    Profit_count = df_log[df_log["Profit"] > 0]["Profit"].count() #แบบไม่หักค่าธรรมเนียม
    Profit_count = df_transaction[df_transaction["Net_Profit"] > 0]["Net_Profit"].count()
    Loss_count = df_log[df_log["Profit"] < 0]["Profit"].count() #แบบไม่หักค่าธรรมเนียม
    Loss_count = df_transaction[df_transaction["Net_Profit"] < 0]["Net_Profit"].count()
    Win_rate = (Profit_count)/(Profit_count+Loss_count) 
    
    Total_Trades = len(df_log)
    
    #Total_Capital_used = df_transaction["Size"] * df_transaction["Price_Open"]
    #Total_Capital_used = total_capital_used.sum()
    
    #Slippage Cost
    df_transaction["Slippage"] = abs(df_transaction["Price_Open"] - df_transaction["Price_Close"])
    Total_Slippage_Cost = df_transaction["Slippage"].sum()
    
    #Market Impact Cost 
    #df_transaction["Market_Impact"] = abs(df_transaction["Price_Open"] - df_transaction["VWAP"])
    #total_market_impact_cost = df_transaction["Market_Impact"].sum()
    #AVG
    #avg_market_impact = df_transaction["Market_Impact"].mean()
   
    
    #Commission & Fee Cost
    total_fee_cost = df_transaction["Fee_Open"].sum() + df_transaction["Fee_Close"].sum()

    #Drawdown Duration ไม่ถูก และยังนึกไม่ออกว่าจะใช้อะไรคำนวณ
    #df_log["Peak_Capital"] = df_log["Capital"].cummax()
    #df_log["Drawdown_Flag"] = df_log["Capital"] < df_log["Peak_Capital"]
    #drawdown_duration = df_log["Drawdown_Flag"].sum()



    
    return {"Date": date, 
            "Final PnL": round(capital - initial_capital, 2), 
            "ROI_IN (%)": round((capital - initial_capital) * 100 / initial_capital, 2), 
            "Final Capital": round(capital, 2), 
            "MDD": round(max_drawdown, 2),
            "Sharpe Ratio" : round(Sharpe_Ratio, 2) ,
            "Profit Factor" : round(ProfitFactor, 2),
            "Win-Rate " : round(Win_rate, 2),
            "Total Trades" : round(Total_Trades, 2),
            "Total Slippage Cost " : round(Total_Slippage_Cost, 2),
            #"Total Market Impact Cost " : round(total_market_impact_cost, 2),
            #"AVG Market Impact " : round(avg_market_impact, 2),
            "Total Fee Cost " : round(total_fee_cost, 2),
            #"Drawdown Duration " : round(drawdown_duration, 2),
           }

# 📌 ประมวลผล
df_results_blind_test = pd.DataFrame([process_file(path, date) for date, path in file_paths.items()])
df_results_blind_test.to_csv("Performance_Report_Blind_Test.csv", index=False)
#df_results_blind_test

In [7]:
df_results_blind_test

,Date,Final PnL,ROI_IN (%),Final Capital,MDD,Sharpe Ratio,Profit Factor,Win-Rate,Total Trades,Total Slippage Cost,Total Fee Cost
0,2024-05-15,2536.15,0.25,1002536.15,344.79,0.09,1.64,0.29,25262,1460.52,19670.75
1,2024-05-16,4548.45,0.45,1004548.45,983.14,0.07,1.63,0.23,37688,2092.02,41484.68


In [8]:
create_csv_viewer("Trade_Logs_Blind_Test")

Output()

In [9]:
create_csv_viewer("Transaction_Pairs_Blind_Test")

Output()